In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
from flapper_sim import FlapperSim

f = FlapperSim(robot_pose=np.array([0, 0, 0.8, 1.57]))
position = f.get_output_measurement()
y_x_dim = position[0]
y_y_dim = position[1]
y_z_dim = position[2]

In [ ]:
position_log = []
target_position_log = []
target_state_log = []
estimated_state_log = []
estimated_state_log_2 = []
u_log = []

In [ ]:
# Discrete time state space model of robot
T = 0.1  # Sampling time

A_d = np.array([[1, 0.1, 0.005],
                [0, 1, 0.1],
                [0, 0, 1]])

B_d = np.array([[0.00016666667], [0.005], [0.1]])

C_d = np.array([[1, 0, 0]])

In [ ]:
# State feedback controller
# F_matrix = 10**3 * np.array([[-1.000000000000000, -0.2, -0.018333333333333]])
# F_matrix = np.array([[-64.000000000000341, -41.600000000000023,  -9.813333333333333]])
# F_matrix = 10**2 * np.array([[-3.52, -1.948,  -0.18673333333333]])
F_matrix = 10**2 * np.array([[-1.879614177380125,  -1.069588791089774,  -0.149871934158914]]) # Fast, meets specs in matlab
# F_matrix = np.array([[-0.861784444348700,  -2.630596657383887,  -2.721911318644770]]) # Super slow  
# F_matrix = np.array([[-5.956242778945807,  -9.261937686008082,  -4.965053452061897]]) # Medium speed (3 poles @ -2)


# State observer
# L = np.array([[-3], [-25], [-100]]) # Deadbeat observer
# L = np.array([[-1.090066116184267], [-4.528759116822354], [-7.739556556092396]])
L = np.array([[-1.529582004419328], [-5.871028337456465], [-9.493798716337043]])
# L = L * 1.2

In [ ]:
# Target path
P0 = np.array([0, 0, 0.8])
# P1 = np.array([0.5, 0.5, 1.2])
P1 = np.array([1, 0, 0.8])

# P1 = np.array([0.75, 0.75, 1.2])
t0 = 0
t1 = 10

In [ ]:
# Target state function
def get_target_x(time):
    if time <= t0:
        output = np.vstack((P0, [0, 0, 0], [0, 0, 0]))
        # print("output.shape: ", output.shape)
        # raise NotImplementedError("Target time is less than t0, not implemented yet")
        return output
    elif time > t1:
        output = np.vstack((P1, [0, 0, 0], [0, 0, 0]))
        # print("output.shape: ", output.shape)
        # raise NotImplementedError("Target time is greater than t1, not implemented yet")
        return output
    elif time <= 0.5 * (t0 + t1):
        # return np.vstack((P0, [0, 0, 0], [0, 0, 0]))
        pos = 2 * (P1 - P0) * ((time - t0) / (t1 - t0)) ** 2 + P0
        vel = 4 * (P1 - P0) * (time - t0) / (t1 - t0) ** 2
        acc = 4 * (P1 - P0) / (t1 - t0) ** 2
        return np.vstack((pos, vel, acc))
    elif time > 0.5 * (t0 + t1):
        # return np.vstack((P1, [0, 0, 0], [0, 0, 0]))
        pos = P1 - 2 * (P1 - P0) * ((time - t1) / (t1 - t0)) ** 2
        vel = -4 * (P1 - P0) * (time - t1) / (t1 - t0) ** 2
        acc = -4 * (P1 - P0) / (t1 - t0) ** 2
        return np.vstack((pos, vel, acc)) # [ [pos_x, pos_y, pos_z], [vel_x, vel_y, vel_z], [acc_x, acc_y, acc_z] ]

In [ ]:
# Test target function
import matplotlib.pyplot as plt

times = np.linspace(0,20)

pos = np.zeros(len(times))
vel = np.zeros(len(times))
acc = np.zeros(len(times))

for i in range(len(times)):
    pos[i] = get_target_x(times[i])[0][0]
    vel[i] = get_target_x(times[i])[1][0]
    acc[i] = get_target_x(times[i])[2][0]

plt.plot(times, pos, 'ro', label='Target X Position')
plt.show()

plt.plot(times, vel, 'go', label='Target X Velocity')
plt.show()

plt.plot(times, acc, 'bo', label='Target X Acceleration')
plt.show()

In [ ]:
# Initial state estimates
x_hat_x_dim = np.array([position[0], [0], [0]])
x_hat_y_dim = np.array([position[1], [0], [0]])
x_hat_z_dim = np.array([position[2], [0], [0]])

In [ ]:
print(x_hat_x_dim.shape)

In [ ]:
y_x_dim = 0
y_y_dim = 0
y_z_dim = 0.8

# Initial state estimates
x_hat_x_dim = np.array([[0], [0], [0]])
x_hat_y_dim = np.array([[0], [0], [0]])
x_hat_z_dim = np.array([[0.8], [0], [0]])

In [ ]:
import time
# Simulation loop
t = 0
target_x = get_target_x(t)

start_time = time.time()

last_position = np.array([0, 0, 0.79])

# for i in range(200):
while time.time() - start_time <= 60:
    t = time.time() - start_time
    position = f.get_output_measurement()
    time.sleep(0.01)  # Simulate the sampling time
    if (position[0] != last_position[0]) or (position[1] != last_position[1]) or (position[2] != last_position[2]):
        last_position = position

        position_log.append(position)
        target_position_log.append(([target_x[0,0]], [target_x[0,1]], [target_x[0,2]]))
        target_state_log.append(target_x)

        y_x_dim = position[0]
        y_y_dim = position[1]
        y_z_dim = position[2]
        # while (time.time() - start_time)*0.9 <= t + 0.1:
        #     time.sleep(0.01)  # Wait for 10 ms to simulate the sampling time
        estimated_state_log.append(([x_hat_x_dim[0, 0]], [x_hat_y_dim[0, 0]], [x_hat_z_dim[0, 0]]))
        estimated_state_log_2.append(np.column_stack((x_hat_x_dim, x_hat_y_dim, x_hat_z_dim)))
        target_x = get_target_x(t)  # Example target positions

        # print("target_x: ", target_x)

        u_x_dim = F_matrix @ (x_hat_x_dim - np.expand_dims(target_x[:, 0], axis=-1))
        u_y_dim = F_matrix @ (x_hat_y_dim - np.expand_dims(target_x[:, 1], axis=-1))
        u_z_dim = F_matrix @ (x_hat_z_dim - np.expand_dims(target_x[:, 2], axis=-1))

        u_x_dim = np.clip(u_x_dim, -0.01, 0.01)
        u_y_dim = np.clip(u_y_dim, -0.01, 0.01)
        u_z_dim = np.clip(u_z_dim, -0.01, 0.01)
        
        print("u_x_dim: ", u_x_dim, " u_y_dim: ", u_y_dim, " u_z_dim: ", u_z_dim)

        # # Force the control inputs to zero for debugging
        # u_x_dim = u_x_dim * 0
        # u_y_dim = u_y_dim * 0
        # u_z_dim = u_z_dim * 0

        x_hat_x_dim = ((A_d + (L @ C_d)) @ x_hat_x_dim) + (B_d * u_x_dim) - (L * y_x_dim)
        x_hat_y_dim = ((A_d + (L @ C_d)) @ x_hat_y_dim) + (B_d * u_y_dim) - (L * y_y_dim)
        x_hat_z_dim = ((A_d + (L @ C_d)) @ x_hat_z_dim) + (B_d * u_z_dim) - (L * y_z_dim)
        # print("x_hat_x_dim: ", x_hat_x_dim, " x_hat_y_dim: ", x_hat_y_dim, " x_hat_z_dim: ", x_hat_z_dim)

        u = np.array([u_x_dim[0, 0], u_y_dim[0, 0], u_z_dim[0, 0]])
        u_log.append(u)
        # print("u: ", u)
        # raise NotImplementedError("Control input u is not correct, expected (3, 1)")
        # f.x[6] = np.array([0.01])
        # f.x[7] = np.array([0])
        # f.x[8] = np.array([0])
        f.step(u=u)

        # y_x_dim = np.array([get_target_x(t)[0, 0]])
        # y_y_dim = np.array([0])
        # y_z_dim = np.array([0.8])

        # position_log.append([y_x_dim, y_y_dim, y_z_dim, 
        #                      np.array([get_target_x(t)[1, 0]]), np.array([0]), np.array([0]),
        #                      np.array([get_target_x(t)[2, 0]]), np.array([0]), np.array([0])])
        # print(position_log[-1])

        # print(estimated_state_log_2[-1])

        # # Zero the measurements for debugging
        # y_x_dim = y_x_dim * 0
        # y_y_dim = y_y_dim * 0
        # y_z_dim = y_z_dim * 0

    # t += T

In [ ]:
# print(np.array(position_log).shape)

In [ ]:
print(target_position_log)

In [ ]:
print(np.array(estimated_state_log_2).shape)

In [ ]:
import plotly.graph_objects as go

# Plot the log of output measurements in 3d
fig = go.Figure()
fig = go.Figure(data=[go.Scatter3d(
    x=np.array(position_log)[:, 0, 0],
    y=np.array(position_log)[:, 1, 0],
    z=np.array(position_log)[:, 2, 0],
    mode='lines+markers',
    marker=dict(size=4),
    line=dict(width=2),
    name='Measured Position'  # Added label
)])
fig.add_trace(go.Scatter3d(
    x=np.array(estimated_state_log)[:, 0, 0],
    y=np.array(estimated_state_log)[:, 1, 0],
    z=np.array(estimated_state_log)[:, 2, 0],
    mode='lines+markers',
    marker=dict(size=4, color='green'),
    line=dict(width=2, color='green'),
    name='Estimated State'  # Added label
))
fig.add_trace(go.Scatter3d(
    x=np.array(target_position_log)[:, 0, 0],
    y=np.array(target_position_log)[:, 1, 0],
    z=np.array(target_position_log)[:, 2, 0],
    mode='lines+markers',
    marker=dict(size=4, color='red'),
    line=dict(width=2, color='red'),
    name='Target Position'  # Added label
))

fig.update_layout(
    title='Measured Position vs Target Position Over Time',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z',
        xaxis=dict(range=[-1.1, 2.1]),
        yaxis=dict(range=[-2.1, 2.1]),
        zaxis=dict(range=[0.3, 1.6])
    ),
    width=800,
    height=600,
    legend=dict(
        x=0.7,
        y=0.1,
        bgcolor="rgba(255, 255, 255, 0.5)"
    )
)
fig.show()

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Convert logs to arrays
position = np.array(position_log)[:, :, 0]
estimated = np.array(estimated_state_log_2)[:, :, 0]
target = np.array(target_position_log)[:, :, 0]

# Create a time index for gradient coloring
time_index = np.arange(position.shape[0])

fig = go.Figure()

# Measured Position — blue gradient
fig.add_trace(go.Scatter3d(
    x=position[:, 0],
    y=position[:, 1],
    z=position[:, 2],
    mode='lines+markers',
    # mode='lines',
    marker=dict(
        size=2,
        color=time_index,
        colorscale='Blues',
        colorbar=dict(title='Time Index'),
    ),
    line=dict(width=2, color='blue'),
    name='Measured Position'
))

# Estimated State — green gradient
fig.add_trace(go.Scatter3d(
    x=estimated[:, 0],
    y=estimated[:, 1],
    z=estimated[:, 2],
    mode='lines+markers',
    # mode='lines',
    marker=dict(
        size=2,
        color=time_index,
        colorscale='Greens',
        showscale=False
    ),
    line=dict(width=2, color='green'),
    name='Estimated State'
))

# Target Position — red gradient
fig.add_trace(go.Scatter3d(
    x=target[:, 0],
    y=target[:, 1],
    z=target[:, 2],
    mode='lines+markers',
    marker=dict(
        size=4,
        color=time_index,
        colorscale='Reds',
        showscale=False
    ),
    line=dict(width=2, color='red'),
    name='Target Position'
))

fig.update_layout(
    title='Measured Position vs Target Position Over Time',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z',
        xaxis=dict(range=[-1.1, 2.1]),
        yaxis=dict(range=[-2.1, 2.1]),
        zaxis=dict(range=[0.3, 1.6])
    ),
    width=800,
    height=600,
    legend=dict(
        x=0.7,
        y=0.1,
        bgcolor="rgba(255, 255, 255, 0.5)"
    )
)

fig.show()


In [ ]:
# Position Plot
import plotly.graph_objects as go

# Plot the log of output measurements in 3d
fig = go.Figure()
fig = go.Figure(data=[go.Scatter3d(
    x=np.array(position_log)[:, 0, 0],
    y=np.array(position_log)[:, 1, 0],
    z=np.array(position_log)[:, 2, 0],
    mode='lines+markers',
    
    marker=dict(size=4,         color=time_index,
        colorscale='Blues',
        showscale=False),
    line=dict(width=2),
    
    name='Measured Position'  # Added label
)])
fig.add_trace(go.Scatter3d(
    x=np.array(estimated_state_log_2)[:, 0, 0],
    y=np.array(estimated_state_log_2)[:, 0, 1],
    z=np.array(estimated_state_log_2)[:, 0, 2],
    mode='lines+markers',
    marker=dict(size=4, color=time_index,
        colorscale='Greens',
        showscale=False),
    line=dict(width=2, color='green'),
    name='Estimated Position'  # Added label
))

fig.update_layout(
    title='Measured Position vs Estimated Position Over Time',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'#,
        # xaxis=dict(range=[-1.1, 2.1]),
        # yaxis=dict(range=[-2.1, 2.1]),
        # zaxis=dict(range=[0.3, 1.6])
    ),
    width=800,
    height=600,
    legend=dict(
        x=0.7,
        y=0.1,
        bgcolor="rgba(255, 255, 255, 0.5)"
    )
)
fig.show()

In [ ]:
# Velocity Plot
import plotly.graph_objects as go

# Plot the log of output measurements in 3d
fig = go.Figure()
fig = go.Figure(data=[go.Scatter3d(
    x=np.array(position_log)[:, 3, 0],
    y=np.array(position_log)[:, 4, 0],
    z=np.array(position_log)[:, 5, 0],
    mode='lines+markers',
    marker=dict(size=4,         color=time_index,
        colorscale='Blues',
        showscale=False),
    line=dict(width=2),
    name='Measured Velocity'  # Added label
)])
fig.add_trace(go.Scatter3d(
    x=np.array(estimated_state_log_2)[:, 1, 0],
    y=np.array(estimated_state_log_2)[:, 1, 1],
    z=np.array(estimated_state_log_2)[:, 1, 2],
    mode='lines+markers',
    marker=dict(size=4,         color=time_index,
        colorscale='Greens',
        showscale=False),
    line=dict(width=2, color='green'),
    name='Estimated Velocity'  # Added label
))

fig.update_layout(
    title='Measured Veloctiy vs Estimated Velocity Over Time',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z',
        xaxis=dict(range=[-1, 1]),
        yaxis=dict(range=[-1, 1]),
        zaxis=dict(range=[-1, 1])
    ),
    width=800,
    height=600,
    legend=dict(
        x=0.7,
        y=0.1,
        bgcolor="rgba(255, 255, 255, 0.5)"
    )
)
fig.show()

In [ ]:
# Acceleration Plot
import plotly.graph_objects as go

# Plot the log of output measurements in 3d
fig = go.Figure()
fig = go.Figure(data=[go.Scatter3d(
    x=np.array(position_log)[:, 6, 0],
    y=np.array(position_log)[:, 7, 0],
    z=np.array(position_log)[:, 8, 0],
    mode='lines+markers',
    marker=dict(size=4,         color=time_index,
        colorscale='Blues',
        showscale=False),
    line=dict(width=2),
    name='Measured Acceleration'  # Added label
)])
fig.add_trace(go.Scatter3d(
    x=np.array(estimated_state_log_2)[:, 2, 0],
    y=np.array(estimated_state_log_2)[:, 2, 1],
    z=np.array(estimated_state_log_2)[:, 2, 2],
    mode='lines+markers',
    marker=dict(size=4,         color=time_index,
        colorscale='Greens',
        showscale=False),
    line=dict(width=2, color='green'),
    name='Estimated Acceleration'  # Added label
))

fig.update_layout(
    title='Measured Acceleration vs Estimated Acceleration Over Time',
    scene=dict(
        xaxis_title='x',
        yaxis_title='y',
        zaxis_title='z'#,
        # xaxis=dict(range=[-1.1, 2.1]),
        # yaxis=dict(range=[-2.1, 2.1]),
        # zaxis=dict(range=[0.3, 1.6])
    ),
    width=800,
    height=600,
    legend=dict(
        x=0.7,
        y=0.1,
        bgcolor="rgba(255, 255, 255, 0.5)"
    )
)
fig.show()

In [ ]:
# plot X position, velocity, and acceleration in 2d
import matplotlib.pyplot as plt
time_dimension = T * np.arange(len(position_log))

plt.plot(time_dimension, np.array(position_log)[:, 0, 0], label='Measured X Position')
plt.plot(time_dimension, np.array(estimated_state_log_2)[:, 0, 0], label='Estimated X Position')
plt.plot(time_dimension, np.array(target_state_log)[:, 0, 0], label='Target X Position')
plt.legend()
plt.show()

plt.plot(time_dimension, np.array(position_log)[:, 3, 0], label='Measured X Velocity')
plt.plot(time_dimension, np.array(estimated_state_log_2)[:, 1, 0], label='Estimated X Velocity')
plt.plot(time_dimension, np.array(target_state_log)[:, 1, 0], label='Target X Velocity')
plt.legend()
plt.show()

plt.plot(time_dimension, np.array(position_log)[:, 6, 0], label='Measured X Acceleration')
plt.plot(time_dimension, np.array(estimated_state_log_2)[:, 2, 0], label='Estimated X Acceleration')
plt.plot(time_dimension, np.array(target_state_log)[:, 2, 0], label='Target X Acceleration')
plt.legend()
plt.show()

plt.plot(time_dimension, np.array(u_log)[:, 0], label='Control Input X')
plt.legend()
plt.show()